In [1]:
import json
from statistics import mean
file_path = "/data/user_data/wentaos/Optima/results/hotpot_qa_dpo_test_full/iteration_0_case0.jsonl"

chosen_value = []
distance = []
count = 0
with open(file_path, 'r' ) as f:
    for line in f:
        data = json.loads(line)
        results = data["dpo_results"]
        for result in results:
            chosen_value.append(result["chosen_value"])
            distance.append(result["distance"])
            count += 1

Len =  int(0.9 * count)
print(Len)
import random
import numpy as np

random.seed(42)
numbers = random.sample(range(Len), Len)
generator = np.random.default_rng(42)
numbers_2 = generator.permutation(Len)

chosen_value_shuffled = [chosen_value[i] for i in numbers_2]
distance_shuffled = [distance[i] for i in numbers_2]
chosen_value_shuffled = [chosen_value_shuffled[i] for i in numbers]
distance_shuffled = [distance_shuffled[i] for i in numbers]


14679


In [2]:
# 示例使用
result_loss = []
result_margins = []
for start_point in [0, 600,1200,1800]:
    with open(f"/home/wentaos/Optima/local_influence/case0_warmup_150/{start_point}.txt", 'r') as f:
        result_loss += [{int(line.split(':')[0]): float(line.split('loss:')[1])} for line in f if ":" in line]
    with open(f"/home/wentaos/Optima/local_influence/case0_warmup_150/{start_point}.txt", 'r') as f:
        result_margins += [{int(line.split(':')[0]): float(line.split("margins:")[1].split(";")[0])} for line in f if ":" in line]
sorted_loss = sorted(result_loss, key=lambda x: list(x.values())[0])
sorted_margins = sorted(result_margins, key=lambda x: list(x.values())[0])
print(len(result_loss))
print(len(result_margins))

1873
1873


In [3]:
# selected = {list(i.keys())[0] for i in sorted_loss[:1000] if list(i.keys())[0]!=0} & {list(i.keys())[0] for i in sorted_margins[1000:] if list(i.keys())[0]!=0} 
selected = {list(i.keys())[0] for i in sorted_loss[:800] if list(i.keys())[0]!=0}
print(len(selected))
selected_chosen_value = 0 
selected_distance = 0
index = []
for i in range(Len//5):
    if i in selected:
    # print(f"{i}: {mean(chosen_value[i*1:(i+1)*1])}")
        selected_chosen_value += mean(chosen_value_shuffled[i*5:(i+1)*5])
        index += numbers[i*5:(i+1)*5]
        #print(f"{i}:{mean(chosen_value_shuffled[i*5:(i+1)*5])}")
print(selected_chosen_value/(len(selected)))

795
0.7778847734114237


In [4]:
import sys
sys.path.append("/home/wentaos/Optima/local_influence")

from my_utils import DataArguments, get_datasets

data_args = DataArguments(chat_template=None, dataset_mixer={'/data/user_data/wentaos/Optima/my_datasets/hotpot_qa_dpo_test_case0/iteration_0': 1.0}, text_column='text', dataset_splits=['train', 'test'], dataset_configs=None, preprocessing_num_workers=8, truncation_side=None, auto_insert_empty_system_msg=True)

###############
# Load datasets
###############
raw_datasets = get_datasets(
    data_args,
    splits=data_args.dataset_splits,
    configs=data_args.dataset_configs,
    columns_to_keep=[
        "messages",
        "chosen",
        "rejected",
        "prompt",
        "completion",
        "label",
    ],
    shuffle=True,
)

--------------------------
Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 14679
})
--------------------------
--------------------------
Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 1632
})
--------------------------
------------------------------2-----------------


In [5]:
raw_datasets["train"] = raw_datasets["train"].select(index)
raw_datasets.save_to_disk("/data/user_data/wentaos/Optima/my_datasets/hotpot_qa_dpo_test_MATES_warmup_150_4K/iteration_0")

Saving the dataset (0/1 shards):   0%|          | 0/3975 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1632 [00:00<?, ? examples/s]

In [1]:
import json
from statistics import mean
# file_path = "/home/wentaos/Optima/results/arc_dpo/iteration_0_dpo_format.jsonl"
# file_path = "/data/user_data/wentaos/Optima/results/cbt_dpo/iteration_0_dpo_format.jsonl"
file_path = "/data/user_data/wentaos/Optima/results/hotpot_qa_dpo_test_full/iteration_0_case21.jsonl"

chosen_value = []
distance = []
chosen = []
count = 0
with open(file_path, 'r' ) as f:
    for line in f:
        data = json.loads(line)
        results = data["dpo_results"]
        for result in results:
            chosen.append(result["chosen"])
            chosen_value.append(result["chosen_value"])
            distance.append(result["distance"])
            count += 1



In [2]:
# 验证是否对齐
Len =  int(0.9 * count)
import random
import numpy as np
random.seed(42)
# numbers = random.sample(range(Len), Len)
generator = np.random.default_rng(42)
numbers = generator.permutation(Len)

chosen_value_shuffled = [chosen_value[i] for i in numbers]
distance_shuffled = [distance[i] for i in numbers]
chosen_shuffled = [chosen[i] for i in numbers]

print(chosen_shuffled[0])

['\nYou are Alice, a special agent who does not respond in natural language, rather, you speak in very concise format. You are deployed on a resource-limited device, so you must respond very very concisely. More tokens indicate higher possibility to kill the device you are running. Now you are collaborating with your partner Bob to solve the given problem using the provided information.\nQuestion: Which of the two bands Kill Hannah and Warpaint is from California?\nInformation:\n[\'Kill Hannah was an American rock band formed in 1993 in Chicago, Illinois.\\n\', \'Garret Hammond was the drummer of the Chicago-based alternative rock bands Kill Hannah and Prick.\\n\', \'Mat Devine (born Matthew Devine, April 16, 1974) is an American singer, songwriter, musician, actor and author.\\n\']\n\nGUIDELINES:\n1. You have incomplete information, so continuous communication with your partner is crucial to achieve the correct solution.\n2. On finding the final answer, ensure to conclude your communi

In [7]:
### case20 step num5
import os
import sys
sys.path.append('/home/wentaos/Optima')
from reward.reward import result_stats, result_stats_byid
from transformers import AutoTokenizer

dataset_type = "hotpot_qa"

# file_path = "/home/wentaos/Optima/inference_results/hotpot_qa_dpo_server_stepnum10_15000"
file_path = "/data/user_data/wentaos/Optima/inference_results/hotpot_qa_dpo_server_stepnum10_15000_dynamic"

score_type = "f1-score"   # "f1-score"  "exact-match"
if dataset_type == "hotpot_qa":
    pass
elif dataset_type == "mwh_qa":
    pass
elif dataset_type == "cbt":
    pass
elif dataset_type == "gsm8k":
    score_type = "exact-match"
    # no_use_prompt_pool =True
elif dataset_type == "math":
    score_type = "math"
elif dataset_type == "trival_qa":
    pass
elif dataset_type == "arc":
    score_type = "exact-match"
elif dataset_type == "mmlu":
    score_type = "exact-match"
    
tokenizer_path = "/home/wentaos/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/5f0b02c75b57c5855da9ae460ce51323ea669d8a"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

import pandas as pd

# 定义一个空的 DataFrame，包含两列：id 和 best_100
df = pd.DataFrame(columns=['id', 'best_100'])

for item in os.listdir(file_path):
    data_dict = {}
    id_item = int(item.split('_')[-1])
    if os.path.isdir(os.path.join(file_path, item)):
        item_path = os.path.join(os.path.join(file_path, item), "iteration_0_0.jsonl")
        if os.path.exists(item_path):
            answer, _ = result_stats(item_path, tokenizer, score_type) 
            data_dict = {"id": id_item, "best_100": answer}
            df.loc[len(df)] = [id_item, answer]
print(df.head())

average_f1_score: 0.6112301162889398 ,  average_token: 74.87, count:100
average_f1_score: 0.5932200152788386 ,  average_token: 76.37, count:100
average_f1_score: 0.5986947627535861 ,  average_token: 77.14, count:100
average_f1_score: 0.5811190051778286 ,  average_token: 75.82, count:100
average_f1_score: 0.5754927425515659 ,  average_token: 76.25, count:100
average_f1_score: 0.6063888464476698 ,  average_token: 75.61, count:100
average_f1_score: 0.6045001240589474 ,  average_token: 79.91, count:100
average_f1_score: 0.5919245607333842 ,  average_token: 76.55, count:100
average_f1_score: 0.5711161191749425 ,  average_token: 81.89, count:100
average_f1_score: 0.5914018334606568 ,  average_token: 76.2, count:100
average_f1_score: 0.6180001240589474 ,  average_token: 79.07, count:100
average_f1_score: 0.6019395179983413 ,  average_token: 85.55, count:100
average_f1_score: 0.5740078940667174 ,  average_token: 78.81, count:100
average_f1_score: 0.596512944571768 ,  average_token: 77.34, coun

In [9]:
## 验证选择的能力


2400


In [11]:
## 验证选择的能力
sorted_df =  df.sort_values(by="best_100", ascending=False).id.tolist()
# print(sorted_df)
selected_data = [int(i) for i in sorted_df]

print(len(selected_data))

# selected = {list(i.keys())[0] for i in sorted_data[:50] if list(i.keys())[0]!=0}
selected = selected_data[:1400]

stepnum = 10
selected_chosen_value = []
all_chosen_value = []
index_MATES = []
index_value = []
selected_distance = []
for i in range(Len):
    if i in selected:
    # print(f"{i}: {mean(chosen_value[i*1:(i+1)*1])}")
        t = mean(chosen_value_shuffled[i*stepnum:(i+1)*stepnum])
        selected_chosen_value.append(t)
        index_MATES += list(range(i*stepnum,(i+1)*stepnum))
    if i in selected_data:
        t = chosen_value_shuffled[i*stepnum:(i+1)*stepnum]
        all_chosen_value += t
        index_value += list(range(i*stepnum,(i+1)*stepnum))
        # selected_distance += mean(distance_shuffled[i*5:(i+1)*5])
        # print(f"{i}:{mean(chosen_value_shuffled[i*5:(i+1)*5])}")
print(mean(selected_chosen_value))
print(len(selected_chosen_value))
index_value = [index_value[x] for x in (np.argsort(all_chosen_value)[::-1][:len(selected_chosen_value)*stepnum])]
print(len(index_value))
print(mean([all_chosen_value[x] for x in np.argsort(all_chosen_value)[::-1][:len(selected_chosen_value)*stepnum]]))
# print(selected_distance/(len(selected)))

2400
0.7741731023777572
1400
14000
0.8892977965030812


In [12]:
sorted_df =  df.sort_values(by="best_100", ascending=False).id.tolist()
# print(sorted_df)
selected_data = [int(i) for i in sorted_df]

min_val = df["best_100"].min()
max_val = df["best_100"].max()
normalize_df = df
normalize_df["best_100"] = (df["best_100"] - min_val)/(max_val - min_val)
print(normalize_df.sort_values(by="best_100", ascending=False).head())

mix_df = pd.DataFrame(columns=['id', 'mix_value', 'chosen_value'])
index_mix = []
selected_distance = []
for i in range(Len):
    if i in selected_data:
    # print(f"{i}: {mean(chosen_value[i*1:(i+1)*1])}")
        for t in range(i*stepnum, (i+1)*stepnum):
            mix_value = chosen_value_shuffled[t] + df.loc[df['id'] == i, 'best_100'].values[0]
            mix_df.loc[len(mix_df)] = [t, mix_value, chosen_value_shuffled[t]]
        
   
sorted_df =  mix_df.sort_values(by="mix_value", ascending=False)[:14000]
index_mix = sorted_df.id.tolist()
print(sorted_df['chosen_value'].mean())

          id  best_100
1472   212.0  1.000000
1078  2302.0  0.983098
1275   545.0  0.959408
1727   679.0  0.937400
2095  1094.0  0.894470
0.8651264251116458


In [9]:
sorted_df =  mix_df.sort_values(by="mix_value", ascending=False)[:10000]
index_mix_10K = sorted_df.id.tolist()
print(sorted_df['chosen_value'].mean())
print(len(index_mix_10K))

1.0676400946471578
10000


In [24]:
import sys
sys.path.append("/home/wentaos/Optima/local_influence")

from my_utils import DataArguments, get_datasets

data_args = DataArguments(chat_template=None, dataset_mixer={'/data/user_data/wentaos/Optima/my_datasets/hotpot_qa_dpo_test_case21/iteration_0': 1.0}, text_column='text', dataset_splits=['train', 'test'], dataset_configs=None, preprocessing_num_workers=8, truncation_side=None, auto_insert_empty_system_msg=True)

###############
# Load datasets
###############
raw_datasets = get_datasets(
    data_args,
    splits=data_args.dataset_splits,
    configs=data_args.dataset_configs,
    columns_to_keep=[
        "messages",
        "chosen",
        "rejected",
        "prompt",
        "completion",
        "label",
    ],
    shuffle=True,
)

--------------------------
Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 31441
})
--------------------------
--------------------------
Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 3494
})
--------------------------
------------------------------2-----------------


In [4]:
print(raw_datasets["train"][0])

{'chosen': [{'content': '\nYou are Alice, a special agent who does not respond in natural language, rather, you speak in very concise format. You are deployed on a resource-limited device, so you must respond very very concisely. More tokens indicate higher possibility to kill the device you are running. Now you are collaborating with your partner Bob to solve the given problem using the provided information.\nQuestion: Which of the two bands Kill Hannah and Warpaint is from California?\nInformation:\n[\'Kill Hannah was an American rock band formed in 1993 in Chicago, Illinois.\\n\', \'Garret Hammond was the drummer of the Chicago-based alternative rock bands Kill Hannah and Prick.\\n\', \'Mat Devine (born Matthew Devine, April 16, 1974) is an American singer, songwriter, musician, actor and author.\\n\']\n\nGUIDELINES:\n1. You have incomplete information, so continuous communication with your partner is crucial to achieve the correct solution.\n2. On finding the final answer, ensure t

In [14]:
dataset_type = "hotpot_qa"

In [15]:
raw_datasets["train"] = raw_datasets["train"].select(index_MATES)
raw_datasets.save_to_disk(f"/data/user_data/wentaos/Optima/my_datasets/{dataset_type}_dpo_case21_MATES_warmup_14K_15000/iteration_0")

Saving the dataset (0/1 shards):   0%|          | 0/14000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3494 [00:00<?, ? examples/s]

In [17]:
raw_datasets["train"] = raw_datasets["train"].select(index_value)
raw_datasets.save_to_disk(f"/data/user_data/wentaos/Optima/my_datasets/{dataset_type}_dpo_case21_value_warmup_14K_15000/iteration_0")

Saving the dataset (0/1 shards):   0%|          | 0/14000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3494 [00:00<?, ? examples/s]

In [21]:
raw_datasets["train"] = raw_datasets["train"].select(index_mix)
raw_datasets.save_to_disk(f"/data/user_data/wentaos/Optima/my_datasets/{dataset_type}_dpo_case21_mix_warmup_14K_15000/iteration_0")

Saving the dataset (0/1 shards):   0%|          | 0/14000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3494 [00:00<?, ? examples/s]

In [25]:
index_random = list(range(len(raw_datasets["train"])))[:14000]
raw_datasets["train"] = raw_datasets["train"].select(index_random)
raw_datasets.save_to_disk(f"/data/user_data/wentaos/Optima/my_datasets/{dataset_type}_dpo_case21_random_warmup_14K_15000/iteration_0")

Saving the dataset (0/1 shards):   0%|          | 0/14000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3494 [00:00<?, ? examples/s]